In [85]:
DATASET = "swedish_tunes_int.csv"
TRANSLATION_FILE = "translation"
INPUT_SIZE = 5
HIDDEN_NODES = 256
NUM_LAYERS = 1
SEED = 0
LEARNING_RATE = 0.1

In [86]:
translation_p = open(TRANSLATION_FILE, 'r')
translation_lines = translation_p.readlines()
num_tokens = len(translation_lines)

translation_p.close()
num_tokens

1178

In [87]:
import pandas
from torch.nn import RNN, Linear, Softmax, Module
import torch
import math

In [88]:
# Class that creates the model that we'll use
class PredictRNN(Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PredictRNN, self).__init__()

        self.rnn = RNN(input_size, hidden_size, num_layers=num_layers)
        self.h2o = Linear(hidden_size, output_size)
        self.softmax = Softmax(dim=0)

    def forward(self, line_tensor):
        rnn_out, hidden = self.rnn(line_tensor)
        output = self.h2o(hidden[0])
        output = self.softmax(output)
        return output

In [89]:
# Get min number of tokens in a line
def trim_small_lines(filepath, input_size):
    file_p = open(filepath, 'r')
    lines = list(file_p.readlines())
    for line in lines:
        line_tok = line.split(',')
        line_len = len(line_tok)
        if line_len < input_size:
            lines.remove(line)
    file_p.close()
    return lines

trim_small_lines(DATASET, INPUT_SIZE)[:5]

['0,1,2,3,4,5,3,6,3,5,3,7,5,5,5,8,5,3,5,4,7,3,4,5,3,6,3,5,3,7,5,5,5,8,9,10,11,12,13,14,13,7,5,5,5,15,15,3,16,3,7,12,13,14,13,7,5,5,5,8,9,17\n',
 '0,1,2,18,19,20,21,22,6,7,23,3,15,23,12,7,6,6,22,24,25,3,12,26,7,27,22,22,21,18,7,13,23,15,27,28,7,29,19,22,23,12,30,31,32,12,12,7,6,20,29,19,33,29,19,20,7,18,19,20,21,22,6,7,23,3,15,23,12,17\n',
 '0,1,2,22,6,23,15,13,18,7,33,29,19,22,34,7,6,6,22,3,15,3,16,13,35,7,24,12,6,23,6,20,7,22,6,23,15,13,18,7,33,29,19,22,34,30,19,36,37,15,6,19,33,7,38,39,39,29,33,7,40,33,29,19,26,38,7,12,20,29,41,17\n',
 '0,1,2,24,4,25,13,3,25,7,3,5,8,23,9,7,6,6,29,4,8,25,5,3,3,3,3,7,8,25,5,15,3,6,6,6,7,8,25,5,3,25,3,3,3,7,3,5,8,23,9,30,12,12,31,32,7,12,42,43,7,24,4,25,13,3,25,7,3,5,8,23,9,17\n',
 '0,1,2,44,45,23,15,7,46,47,48,47,12,7,6,20,49,50,51,46,52,46,50,46,7,6,25,15,3,7,46,47,48,5,8,7,47,52,50,53,24,30,54,45,3,20,7,3,20,19,20,7,44,45,23,15,7,46,47,48,47,12,17\n']

In [90]:
# calculates the accuracy of the predictions of a neural network
# For classification tasks
# NOTE: X and y should already be PyTorch tensors
def calculate_accuracy(network, X, y):
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        X = X.to(device)
        y = y.to(device)
    # make predictions for the given X
    probs = network(X)
    predictions = torch.argmax(probs, dim=1)
    # the calculate accuracy of those predictions
    total = len(predictions)
    accuracy = sum(predictions == y) / total
    variance = accuracy * (1 - accuracy)
    std_err = math.sqrt(variance / total)
    up_bound = accuracy + 2.39 * std_err
    low_bound = accuracy - 2.39 * std_err
    return float(accuracy), up_bound, low_bound


In [91]:
# converts a training set into smaller train and validation sets
def create_validation(training_X, training_y, valid_percentage):
    # find the split point between training and validation
    training_n = training_X.shape[0]
    valid_rows = int(valid_percentage * training_n)

    # create the validation set
    valid_X = training_X.iloc[:valid_rows]
    valid_y = training_y.iloc[:valid_rows]

    # create the (smaller) training set
    train_X = training_X.iloc[valid_rows:]
    train_y = training_y.iloc[valid_rows:]

    return train_X, train_y, valid_X, valid_y


In [92]:
# trains a neural network with given training data
def train_network(network, training_X, training_y, lr, nn_filepath):
    # split the training data into train and validation
    # Note: use 20% of the original training data for validation
    train_X, train_y, valid_X, valid_y = create_validation(training_X, training_y, 0.2)

    # convert our data to PyTorch objects
    train_X = torch.from_numpy(train_X.values).float()
    train_y = torch.from_numpy(train_y.values).long()
    valid_X = torch.from_numpy(valid_X.values).float()
    valid_y = torch.from_numpy(valid_y.values).long()

    # move the data and model to the GPU if possible
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print(f"Using device {torch.cuda.get_device_name(device)}")
        train_X = train_X.to(device)
        train_y = train_y.to(device)
        valid_X = valid_X.to(device)
        valid_y = valid_y.to(device)
        network = network.to(device)

    # create the algorithm that learns the weight for the network
    optimizer = torch.optim.Adam(network.parameters(), lr=lr)

    # create the loss function that tells optimizer how much error it has in its predictions
    # here we use cross entropy since we have a classification task with more than two possible labels
    loss_function = torch.nn.CrossEntropyLoss()

    # train for 1000 epochs
    num_epochs = 1000
    train_loss_values = []
    valid_loss_values = []
    train_acc_values = []
    valid_acc_values = []
    for epoch in range(num_epochs):
        # network.zero_grad()
        # # make predictions on the training set and validation set
        # for X_vec, y_label in zip(train_X, train_y):
        #     prediction = network(X_vec)
        #     loss = loss_function(prediction, y_label)
        train_predictions = network(train_X)
        valid_predictions = network(valid_X)
        train_loss = loss_function(train_predictions, train_y)

        # calculate the error on the training set
        train_loss_values.append(train_loss.item())
        valid_loss_values.append(loss_function(valid_predictions, valid_y).item())
        train_acc_values.append(calculate_accuracy(network, train_X, train_y))
        valid_acc = calculate_accuracy(network, valid_X, valid_y)
        valid_acc_values.append(valid_acc)

        # Early return for perfect fit
        if valid_acc == 1:
            # convert the training progress data to a Pandas DataFrame
            progress = {
                "epoch": range(epoch+1),
                "train_loss": train_loss_values,
                "valid_loss": valid_loss_values,
                "train_acc": train_acc_values,
                "valid_acc": valid_acc_values
            }
            torch.save(network, nn_filepath)
            return pandas.DataFrame(progress)

        # perform backpropagation
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # convert the training progress data to a Pandas DataFrame
    progress = {
        "epoch": range(num_epochs),
        "train_loss": train_loss_values,
        "valid_loss": valid_loss_values,
        "train_acc": train_acc_values,
        "valid_acc": valid_acc_values
    }
    torch.save(network, nn_filepath)
    return pandas.DataFrame(progress)


In [93]:
def process_dataset(dataset_filepath, input_size):
    lines = trim_small_lines(dataset_filepath, input_size)
    # We need to create a dataframe with the proper number of columns
    # Get the first three tokens from every line and store them in front of each instance
    dataframe_dict = {
        'timesig': [],
        'key': [],
        'style': []
    }

    for i in range(input_size):
        dataframe_dict[i] = []

    for line in lines:
        line = line.split(',')
        timesig = int(line[0])
        key = int(line[1])
        style = int(line[2])
        remainder = list(map(int, line[3:]))
        for i in range(len(remainder) - input_size):
            window = remainder[i:i+input_size]
            dataframe_dict['timesig'].append(timesig)
            dataframe_dict['key'].append(key)
            dataframe_dict['style'].append(style)
            for ind, token in enumerate(window):
                dataframe_dict[ind].append(token)

    return pandas.DataFrame(dataframe_dict)

process_dataset(DATASET, INPUT_SIZE)

,timesig,key,style,0,1,2,3,4
0,0,1,2,3,4,5,3,6
1,0,1,2,4,5,3,6,3
2,0,1,2,5,3,6,3,5
3,0,1,2,3,6,3,5,3
4,0,1,2,6,3,5,3,7
...,...,...,...,...,...,...,...,...
901002,0,1,800,35,14,7,43,27
901003,0,1,800,14,7,43,27,27
901004,0,1,800,7,43,27,27,7
901005,0,1,800,43,27,27,7,76


In [94]:
# Splits a dataset into training and test sets,
def split_dataset(dataset, train_data_ratio, y_label, seed=SEED):
    training = dataset.sample(frac=train_data_ratio, random_state=seed)
    test = dataset.drop(training.index)
    training_y = training[y_label]
    training_X = training.drop(columns=y_label)
    test_y = test[y_label]
    test_X = test.drop(columns=y_label)
    return training_X, training_y, test_X, test_y

dataset = process_dataset(DATASET, INPUT_SIZE)
training_X, training_y, test_X, test_y = split_dataset(dataset, 0.9, 4, 0)
training_X


,timesig,key,style,0,1,2,3
119510,0,1,575,42,13,7,14
31816,64,1,274,121,79,11,20
898913,0,179,800,6,7,193,7
295356,128,179,172,25,7,163,55
531065,128,1,172,55,212,55,7
...,...,...,...,...,...,...,...
215929,128,179,720,16,57,16,5
712481,128,1,725,7,88,22,12
351176,128,179,172,5,187,188,7
589603,128,179,172,187,15,23,15


In [95]:
# Performs one-hot encoding on dataset AFTER process_dataset
# Napkin estimates this would take 27 hours to run on swedish dataset
def onehots(dataframe, n_tokens):
    ret_frame = pandas.DataFrame()
    for column_name, column in dataframe.items():
        oh_dict = {}
        for ix in range(n_tokens):
            oh_dict[str(column_name) + '_' + str(ix)] = []
        for item in column:
            print(item)
            for key, array in oh_dict.items():
                if item == int(key.split('_')[1]):
                    array.append(1)
                else:
                    array.append(0)
        ret_frame = pandas.concat([ret_frame, pandas.DataFrame(oh_dict)], axis=1)
    return ret_frame

In [96]:
rnn = PredictRNN(INPUT_SIZE+2, HIDDEN_NODES, NUM_LAYERS, num_tokens)
results = train_network(rnn, training_X, training_y, LEARNING_RATE, 'trained_rnn.pt')
test_X = torch.from_numpy(test_X.values).float()
test_y = torch.from_numpy(test_y.values).long()
accuracy, up_bound, low_bound = calculate_accuracy(rnn, test_X, test_y)
print(f"Accuracy: {accuracy:.3f}")
print(f"95% Confidence Interval: [{low_bound:.3f}, {up_bound:.3f}]")


Using device AMD Radeon RX 7900 XT
training_X
torch.Size([648725, 7])
train_predictions
torch.Size([1178])
train_y
torch.Size([648725])


RuntimeError: size mismatch (got input: [1178], target: [648725])